# İdrar Sedimenti Görüntü Analizi - Keşifsel Analiz

Bu notebook sentetik veri setini keşfetmek ve model performansını analiz etmek için kullanılabilir.


In [ ]:
import sys
sys.path.append('../')

import torch
from torch.utils.data import DataLoader
from src.data.dataset import UrineFieldDataset
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image


In [ ]:
# Veri setini yükle
dataset = UrineFieldDataset('../data/synthetic', train=False)
print(f"Toplam görüntü sayısı: {len(dataset)}")
print(f"Sınıflar: {dataset.get_class_names()}")


In [ ]:
# Sınıf dağılımını görselleştir
from collections import Counter

labels = [dataset[i][1] for i in range(len(dataset))]
label_counts = Counter(labels)

plt.figure(figsize=(10, 6))
plt.bar(range(len(label_counts)), list(label_counts.values()))
plt.xlabel('Sınıf')
plt.ylabel('Görüntü Sayısı')
plt.title('Sınıf Dağılımı')
plt.xticks(range(len(label_counts)), dataset.get_class_names())
plt.show()


In [ ]:
# Örnek görüntüleri göster
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Her sınıf için örnekleri bul
class_indices_dict = {}
for i in range(len(dataset)):
    try:
        _, label = dataset[i]
        if label not in class_indices_dict:
            class_indices_dict[label] = []
        class_indices_dict[label].append(i)
    except Exception as e:
        print(f"Veri yüklenirken hata (index {i}): {e}")
        continue

for cls_idx in range(4):
    # Her sınıftan 2 örnek
    if cls_idx in class_indices_dict:
        indices = class_indices_dict[cls_idx][:2]
    else:
        indices = []
    
    for sample_idx in range(2):
        if sample_idx < len(indices):
            try:
                idx = indices[sample_idx]
                img, label = dataset[idx]
                
                # Denormalize (ImageNet normalization'ı geri al)
                img_np = img.permute(1, 2, 0).numpy()
                img_np = img_np * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
                img_np = np.clip(img_np, 0, 1)
                
                axes[sample_idx, cls_idx].imshow(img_np)
                axes[sample_idx, cls_idx].set_title(f"{dataset.get_class_names()[label]}")
                axes[sample_idx, cls_idx].axis('off')
            except Exception as e:
                print(f"Görüntü gösterilirken hata (cls={cls_idx}, sample={sample_idx}): {e}")
                axes[sample_idx, cls_idx].axis('off')
        else:
            axes[sample_idx, cls_idx].axis('off')

plt.tight_layout()
plt.show()
